In [471]:
#####Modelling Reciprocating Relationships with Hawkes Processes#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [472]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    return z_id

In [473]:
####データの発生####
##データの設定
k1 = 9
k2 = 7
hh = 5000
item = 3000
pt = np.random.poisson(np.random.gamma(20.0, 1/0.25, hh), hh); pt[pt <= 5] = 5
hhpt = np.sum(pt)

In [474]:
##IDとインデックスを設定
#IDを設定
d_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスを設定
d_list = [i for i in range(hh)]
d_vec = [i for i in range(hh)]
pt_list = [j for j in range(np.max(pt))]
pt_n = np.repeat(0, np.max(pt))
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
    d_vec[i] = np.repeat(1, pt[i])
for j in range(np.max(pt)):
    pt_list[j] = np.array(np.where(pt_id==j)[0], dtype="int")
    pt_n[j] = pt_list[j].shape[0]
max_index = np.array([np.max(d_list[i]) for i in range(hh)])

In [475]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[d_list[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_list = [j for j in range(item)]
item_vec = [j for j in range(item)]
item_n = np.repeat(0, item)
for j in range(item):
    item_list[j] = np.array(np.where(item_id==j)[0], dtype="int")
    item_vec[j] = np.repeat(1, len(item_list[j]))
    item_n[j] = len(item_list[j])

0
1000
2000
3000
4000


In [476]:
##応答変数が妥当な値になるまで繰り返す
rp = 0
while True:
    rp = rp + 1

    ##パラメータを生成
    #潜在変数を生成
    theta1 = np.random.dirichlet(np.repeat(2.0, k1), 1).reshape(-1)
    theta2 = np.random.dirichlet(np.repeat(2.0, k2), 1).reshape(-1)
    Z1 = np.random.multinomial(1, theta1, hh)
    Z2 = np.random.multinomial(1, theta2, item)
    z1_vec = np.dot(Z1, np.arange(k1)); z2_vec = np.dot(Z2, np.arange(k2))
    thetat1 = theta1.copy(); thetat2 = theta2.copy()
    
    #ガンマ分布の事前分布のパラメータ
    alpha1 = 0.75; beta1 = 0.15
    alpha2 = 0.75; beta2 = 0.125
    alpha3 = 0.5; beta3 = 0.125

    #モデルパラメータを生成
    gamma1 = np.random.gamma(alpha1, 1/beta1, k1)
    gamma2 = np.random.gamma(alpha2, 1/beta2, k2)
    gamma3 = np.random.gamma(alpha3, 1/beta3, k1*k2).reshape(k1, k2)
    gammat1 = gamma1.copy(); gammat2 = gamma2.copy(); gammat3 = gamma3.copy()

    ##応答変数を生成
    #モデルの期待値
    mu1 = gamma1[z1_vec[d_id], ]
    mu2 = gamma2[z2_vec[item_id], ]
    mu3 = np.sum(gamma3[z1_vec[d_id], ] * Z2[item_id, ], axis=1)
    mu = mu1 + mu2 + mu3
    mut = mu.copy()
    
    #指数分布から応答変数を生成
    y = np.random.exponential(mu, hhpt)

    #break条件
    print([rp, np.max(y), np.min(y)])
    if (np.max(y) < 200):
        break

[1, 368.562512609788, 7.335296061241956e-05]
[2, 344.1217025867494, 2.770719500733708e-06]
[3, 334.1702696148389, 4.886100183555321e-05]
[4, 386.00514448660317, 1.953856934259078e-05]
[5, 498.7948463629506, 6.170661136864558e-06]
[6, 305.41173291881876, 2.0889439015682148e-05]
[7, 500.62295440786954, 2.034316822472179e-05]
[8, 416.091546286761, 2.3152943050962185e-05]
[9, 496.7560077792393, 6.613528917705023e-05]
[10, 343.3990237827278, 4.032440956917208e-05]
[11, 305.04485987710063, 3.249843754663835e-07]
[12, 460.6486620972208, 1.4214634694028713e-05]
[13, 438.9463265006692, 2.3348754770165273e-05]
[14, 579.4466111220848, 6.596301299862725e-06]
[15, 298.39243821030215, 1.152178379421791e-05]
[16, 223.54986061881166, 2.7457371579072623e-06]
[17, 654.2352250733487, 1.8211593957750522e-05]
[18, 452.01001163251493, 1.9057234151438426e-05]
[19, 314.64797580753844, 2.9500008839303437e-05]
[20, 497.9327588600854, 9.983886664313713e-06]
[21, 576.8470459580236, 2.245993933332974e-05]
[22, 205

In [477]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断指数分布の乱数を生成する関数
def rtexp(gamma, a, b):
    #切断指数分布の乱数を生成
    FA = scipy.stats.expon.cdf(a, scale=gamma)
    FB = scipy.stats.expon.cdf(b, scale=gamma)
    par = scipy.stats.expon.ppf(np.random.uniform(0, 1, a.shape[0])*(FB-FA)+FA, scale=gamma)
    return par

In [478]:
##データの設定
R = 2000
keep = 4
burnin = int(500/keep)
iter = 0
disp = 10
e1 = 0.001
e2 = 0.0025
L = 3

In [479]:
##事前分布の設定
alpha01 = 1.0; beta01 = 1.0
alpha02 = 1.0; beta02 = 1.0
alpha03 = 1.0; beta03 = 1.0
alpha11 = 1.0
alpha12 = 1.0

In [485]:
##パラメータの真値
#潜在変数の真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()
Zi1 = Z1.copy()
Zi2 = Z2.copy()
z1_vec = np.dot(Zi1, np.arange(k1))
z2_vec = np.dot(Zi2, np.arange(k2))

#モデルパラメータの真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()
gamma1 = gammat1.copy()
gamma2 = gammat2.copy()
gamma3 = gammat3.copy()

#モデルの期待値
mu1 = gamma1[z1_vec[d_id]]
mu2 = gamma2[z2_vec[item_id]]
mu3 = np.sum(gamma3[z1_vec[d_id], ] * Z2[item_id, ], axis=1)

In [486]:
##パラメータの初期値
#潜在変数の初期値
theta1 = np.random.dirichlet(np.repeat(5.0, k1), 1).reshape(-1)
theta2 = np.random.dirichlet(np.repeat(5.0, k2), 1).reshape(-1)
Zi1 = np.random.multinomial(1, theta1, hh)
Zi2 = np.random.multinomial(1, theta2, item)
z1_vec = np.dot(Zi1, np.arange(k1))
z2_vec = np.dot(Zi2, np.arange(k2))

#モデルパラメータの初期値
a = np.mean(y) / 3.0; b = 1.0
gamma1 = np.random.gamma(a, 1/b, k1)
gamma2 = np.random.gamma(a, 1/b, k2)
gamma3 = np.random.gamma(a, 1/b, k1*k2).reshape(k1, k2)

In [487]:
##パラメータの格納用配列
#潜在変数の格納用配列
SEG1 = np.zeros((hh, k1))
SEG2 = np.zeros((item, k2))

#モデルパラメータの格納用配列
GAMMA1 = np.zeros((int(R/keep), k1))
GAMMA2 = np.zeros((int(R/keep), k2))
GAMMA3 = np.zeros((k1, k2, int(R/keep)))

In [488]:
##対数尤度の基準値
#1パラメータモデルの対数尤度
LLst = np.sum(scipy.stats.expon.logpdf(x=y, scale=np.mean(y)))
print(LLst)

#真値での対数尤度
LLbest = np.sum(scipy.stats.expon.logpdf(x=y, scale=mut))
print(LLbest)

-1256845.3255435547
-1150606.545826656


In [490]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##ユーザーの潜在変数をサンプリング
    #潜在変数の割当確率
    LLho = np.zeros((hhpt, k1)); Lho = np.zeros((hh, k1))
    Lambda = np.full((hhpt, k1), gamma1) + mu2[:, np.newaxis] + gamma3.T[z2_vec[item_id], ]
    for j in range(k1):
        LLho[:, j] = scipy.stats.expon.logpdf(x=y, scale=Lambda[:, j])
    for i in range(hh):
        Lho[i, ] = np.sum(LLho[d_list[i], ], axis=0)
    Lho = theta1 * np.exp(Lho - np.max(Lho, axis=1)[:, np.newaxis])
    Prob = Lho / np.sum(Lho, axis=1)[:, np.newaxis]

    #多項分布から潜在変数をサンプリング
    res = rmnom(Prob, hh, k1, np.arange(hh), 1)
    z1_vec = np.array(res[0], dtype="int")
    Zi1 = np.array(res[1].todense(), dtype="int")
    z1 = z1_vec[d_id]
    mu1 = gamma1[z1_vec[d_id]]

    ##アイテムの潜在変数をサンプリング
    #潜在変数の割当確率
    LLho = np.zeros((hhpt, k2)); Lho = np.zeros((item, k2))
    Lambda = mu1[:, np.newaxis] + np.full((hhpt, k2), gamma2) + gamma3[z1_vec[d_id], ]
    for j in range(k2):
        LLho[:, j] = scipy.stats.expon.logpdf(x=y, scale=Lambda[:, j])   
    for i in range(item):
        Lho[i, ] = np.sum(LLho[item_list[i], ], axis=0)
    Lho = theta2 * np.exp(Lho - np.max(Lho, axis=1)[:, np.newaxis])
    Prob = Lho / np.sum(Lho, axis=1)[:, np.newaxis]

    #多項分布から潜在変数をサンプリング
    res = rmnom(Prob, item, k2, np.arange(item), 1)
    z2_vec = np.array(res[0], dtype="int")
    Zi2 = np.array(res[1].todense(), dtype="int")
    z2 = z2_vec[item_id]
    mu2 = gamma2[z2_vec[item_id]]


    ##ガンマ分布よりユーザーパラメータをサンプリング
    #補助変数lambdaを更新
    mu = mu1 + mu2 + mu3
    Lambda = mu1 / mu

    #事後分布のパラメータを更新
    lambda_y = Lambda * y
    s1 = np.repeat(0.0, k1); v1 = np.repeat(0.0, k1)
    for j in range(k1):
        index = np.where(z1==j)[0]
        s1[j] = np.sum(lambda_y[index]) + alpha01
        v1[j] = np.sum(index.shape[0]) + beta01

    #パラメータをサンプリング
    gamma1 = np.random.gamma(s1, 1/v1, k1)
    mu1 = gamma1[z1_vec[d_id]]

    ##ガンマ分布よりアイテムパラメータをサンプリング
    #補助変数lambdaを更新
    mu = mu1 + mu2 + mu3
    Lambda = mu2 / mu

    #事後分布のパラメータを更新
    lambda_y = Lambda * y
    s2 = np.repeat(0.0, k2); v2 = np.repeat(0.0, k2)
    for j in range(k2):
        index = np.where(z2==j)[0]
        s2[j] = np.sum(lambda_y[index]) + alpha02
        v2[j] = np.sum(index.shape[0]) + beta02

    #パラメータをサンプリング
    gamma2 = np.random.gamma(s2, 1/v2, k2)
    mu2 = gamma2[z2_vec[item_id]]

    ##ガンマ分布よりユーザー-アイテムパラメータをサンプリング
    #補助変数lambdaを更新
    mu = mu1 + mu2 + mu3
    Lambda = (mu3 / mu)

    #事後分布のパラメータを更新
    temp_z1 = Zi1[d_id, ]; temp_z2 = Zi2[item_id, ]
    lambda_y = (Lambda * y)[:, np.newaxis]
    for j in range(k2):
        temp_z = temp_z1 * temp_z2[:, j][:, np.newaxis]
        s3 = np.sum(lambda_y * temp_z, axis=0) + alpha03
        v3 = np.sum(temp_z, axis=0) + beta03

        #パラメータをサンプリング
        gamma3[:, j] = np.random.gamma(s3, 1/v3, k1)
    mu3 = np.sum(gamma3[z1_vec[d_id], ] * Z2[item_id, ], axis=1)

    ##ディリクレ分布から混合率を更新
    theta1 = np.random.dirichlet(np.sum(Zi1, axis=0) + alpha11, 1).reshape(-1)
    theta2 = np.random.dirichlet(np.sum(Zi2, axis=0) + alpha12, 1).reshape(-1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        GAMMA1[mkeep, ] = gamma1
        GAMMA2[mkeep, ] = gamma2
        GAMMA3[:, :, mkeep] = gamma3
        if rp >= burnin:
            SEG1 = SEG1 + Zi1
            SEG2 = SEG2 + Zi2

    if rp%disp==0:
        #対数尤度を更新
        mu = mu1 + mu2 + mu3
        LL = np.sum(scipy.stats.expon.logpdf(x=y, scale=mu))

        #サンプリング結果の表示
        print(rp)
        print(np.round(np.array([LL, LLst, LLbest]), 1))

0
[-1191217.5 -1256845.3 -1150606.5]
10
[-1190668.9 -1256845.3 -1150606.5]
20
[-1191268.5 -1256845.3 -1150606.5]
30
[-1189435.5 -1256845.3 -1150606.5]
40
[-1193514.8 -1256845.3 -1150606.5]
50
[-1193985.3 -1256845.3 -1150606.5]
60
[-1286539.2 -1256845.3 -1150606.5]
70
[-1184751.4 -1256845.3 -1150606.5]
80
[-1184222.3 -1256845.3 -1150606.5]
90
[-1183205.  -1256845.3 -1150606.5]
100
[-1183379.4 -1256845.3 -1150606.5]
110
[-1183331.2 -1256845.3 -1150606.5]
120
[-1182952.8 -1256845.3 -1150606.5]
130
[-1181955.8 -1256845.3 -1150606.5]
140
[-1182083.7 -1256845.3 -1150606.5]
150
[-1184411.3 -1256845.3 -1150606.5]
160
[-1186675.2 -1256845.3 -1150606.5]
170
[-1187904.9 -1256845.3 -1150606.5]
180
[-1188506.7 -1256845.3 -1150606.5]
190
[-1187367.6 -1256845.3 -1150606.5]
200
[-1187781.4 -1256845.3 -1150606.5]
210
[-1187707.3 -1256845.3 -1150606.5]
220
[-1187604.5 -1256845.3 -1150606.5]
230
[-1188028.6 -1256845.3 -1150606.5]
240
[-1187165.8 -1256845.3 -1150606.5]
250
[-1187874.6 -1256845.3 -1150606.